In [1]:
%load_ext autoreload
%autoreload

## This expands a notebook to full width
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Show Python Version
import sys
print("Python: {0}".format(sys.version))

## Show Current Time
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]
Notebook Last Run Initiated: 2023-01-20 22:40:04.500046


## Setup

In [2]:
import numpy as np
np.random.seed(seed=46)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
import tensorflow_addons as tfa
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from PIL import Image
from PIL import ImageEnhance

from random import randrange

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  1
2.9.0


In [5]:
def get_filepaths(directory):
    
    file_paths = []  # List which will store all of the full filepaths.
    file_image = []  # Image filename
   
    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
                       
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.       
            file_image.append(filename.split('.')[0])

    return file_paths, file_image 

In [6]:
class DataGenerator(tf.keras.utils.Sequence):
        
    def __init__(self, df_X, arr_Y, batch_size=32,shuffle=False,image_size=128,mode='train'):
        self.batch_size = batch_size
        self.df_X = df_X
        self.arr_Y = arr_Y
        self.indices = self.df_X.index.tolist()
        self.shuffle = shuffle
        self.image_size = image_size
        self.mode = mode
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X, y = self.__get_data(batch)
        return X, y
    
    def n(self):
        return len(self.indices)
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        X = []
        y = []
               
        for i, id in enumerate(batch):
                       
            # Data
            file = self.df_X.iloc[self.indices[id],0]
            img = load_img(file,color_mode='rgb', target_size=(self.image_size,self.image_size),interpolation='nearest')
            img = img_to_array(img).astype(np.float32)

            if self.mode == 'train':
                img = tf.image.random_flip_left_right(img)
                img = tf.image.random_flip_up_down(img)
            
            X.append(img)
            y.append(self.arr_Y[self.indices[id]])
            
        return np.array(X), np.array(y).reshape(self.batch_size,1)

## Prepare the data

In [7]:
data = pd.read_csv('train.csv')
data = data.loc[data.bolts <= 5]

In [8]:
data.shape

(513, 4)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, data.bolts.values, test_size=0.20, random_state=1)

In [10]:
X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)

## Configure the hyperparameters

In [11]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 1000
image_size = 128  # We'll resize input images to this size
patch_size = 16  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 6
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 24
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

num_classes = 4
input_shape = (128, 128, 3)

## Implement multilayer perceptron (MLP)

In [12]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

## Implement patch creation as a layer

In [13]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

## Implement the patch encoding layer

The `PatchEncoder` layer will linearly transform a patch by projecting it into a
vector of size `projection_dim`. In addition, it adds a learnable position
embedding to the projected vector.

In [14]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


## Build the ViT model

The ViT model consists of multiple Transformer blocks,
which use the `layers.MultiHeadAttention` layer as a self-attention mechanism
applied to the sequence of patches. The Transformer blocks produce a
`[batch_size, num_patches, projection_dim]` tensor, which is processed via an
classifier head with softmax to produce the final class probabilities output.

Unlike the technique described in the [paper](https://arxiv.org/abs/2010.11929),
which prepends a learnable embedding to the sequence of encoded patches to serve
as the image representation, all the outputs of the final Transformer block are
reshaped with `layers.Flatten()` and used as the image
representation input to the classifier head.
Note that the `layers.GlobalAveragePooling1D` layer
could also be used instead to aggregate the outputs of the Transformer block,
especially when the number of patches and the projection dimensions are large.

In [15]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


## Compile, train, and evaluate the model

In [16]:
def run_experiment(model):
        
    model.trainable = False
    set_trainable = False

    for layer in model.layers:
        if layer.name == 'layer_normalization_96':
            set_trainable = True
        
        if set_trainable == True:
            layer.trainable = True
        else:
            layer.trainable = False    
            
    logits = layers.Dense(1)(model.layers[-1].output)

    model2 = keras.Model(inputs=model.inputs, outputs=[logits])
        
    tf.random.set_seed(46)
    
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

    model2.compile(optimizer=optimizer,loss=tf.keras.losses.MeanSquaredError())

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=20, restore_best_weights=True)
    
    checkpoint_filepath = "tmp/checkpoint_ViT.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
        mode='min'
    )

    train_generator = DataGenerator(df_X=X_train, arr_Y=y_train, batch_size=batch_size, shuffle=True,mode='trainx')
    test_generator = DataGenerator(df_X=X_test, arr_Y=y_test, batch_size=batch_size,mode='test')
    
    STEP_SIZE_TRAIN = train_generator.n()//train_generator.batch_size
    STEP_SIZE_VALID = test_generator.n()//test_generator.batch_size
    
    history = model2.fit(x=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=test_generator,
                        validation_steps=STEP_SIZE_VALID, 
                        batch_size=batch_size,
                        epochs=num_epochs,
                        callbacks=[checkpoint_callback,early_stopping],
                        verbose=2)

    model2.load_weights(checkpoint_filepath)
    loss = model2.evaluate(x=test_generator,steps=STEP_SIZE_VALID)
    print(f"loss : {loss}")

    return model2

In [17]:
vit_classifier = create_vit_classifier()
vit_classifier.load_weights('tmp/checkpoint_P00.h5')

In [18]:
vit_classifier = run_experiment(vit_classifier)

Epoch 1/1000
12/12 - 18s - loss: 7151.9238 - val_loss: 4673.4761 - 18s/epoch - 2s/step
Epoch 2/1000
12/12 - 2s - loss: 6552.4194 - val_loss: 4102.8887 - 2s/epoch - 149ms/step
Epoch 3/1000
12/12 - 2s - loss: 5927.7383 - val_loss: 3593.8289 - 2s/epoch - 150ms/step
Epoch 4/1000
12/12 - 2s - loss: 5208.2236 - val_loss: 3123.7917 - 2s/epoch - 159ms/step
Epoch 5/1000
12/12 - 2s - loss: 4747.0103 - val_loss: 2722.9446 - 2s/epoch - 165ms/step
Epoch 6/1000
12/12 - 2s - loss: 3937.3601 - val_loss: 2370.0745 - 2s/epoch - 164ms/step
Epoch 7/1000
12/12 - 2s - loss: 3311.4775 - val_loss: 2075.8816 - 2s/epoch - 169ms/step
Epoch 8/1000
12/12 - 2s - loss: 3269.7654 - val_loss: 1833.7280 - 2s/epoch - 154ms/step
Epoch 9/1000
12/12 - 2s - loss: 3029.6172 - val_loss: 1612.5287 - 2s/epoch - 152ms/step
Epoch 10/1000
12/12 - 2s - loss: 2600.7798 - val_loss: 1430.8340 - 2s/epoch - 151ms/step
Epoch 11/1000
12/12 - 2s - loss: 2312.6384 - val_loss: 1279.9667 - 2s/epoch - 176ms/step
Epoch 12/1000
12/12 - 2s - loss

Epoch 95/1000
12/12 - 2s - loss: 178.5419 - val_loss: 60.1177 - 2s/epoch - 153ms/step
Epoch 96/1000
12/12 - 2s - loss: 187.3310 - val_loss: 57.7288 - 2s/epoch - 155ms/step
Epoch 97/1000
12/12 - 2s - loss: 148.9795 - val_loss: 55.4227 - 2s/epoch - 158ms/step
Epoch 98/1000
12/12 - 2s - loss: 124.8167 - val_loss: 53.3330 - 2s/epoch - 154ms/step
Epoch 99/1000
12/12 - 2s - loss: 122.9478 - val_loss: 51.4448 - 2s/epoch - 156ms/step
Epoch 100/1000
12/12 - 2s - loss: 108.7251 - val_loss: 49.8108 - 2s/epoch - 159ms/step
Epoch 101/1000
12/12 - 2s - loss: 116.9273 - val_loss: 48.1365 - 2s/epoch - 147ms/step
Epoch 102/1000
12/12 - 2s - loss: 105.4384 - val_loss: 46.6047 - 2s/epoch - 157ms/step
Epoch 103/1000
12/12 - 2s - loss: 114.9048 - val_loss: 44.7525 - 2s/epoch - 171ms/step
Epoch 104/1000
12/12 - 2s - loss: 105.7385 - val_loss: 43.1529 - 2s/epoch - 152ms/step
Epoch 105/1000
12/12 - 2s - loss: 101.8482 - val_loss: 41.7083 - 2s/epoch - 157ms/step
Epoch 106/1000
12/12 - 2s - loss: 101.6909 - val

Epoch 191/1000
12/12 - 2s - loss: 5.8617 - val_loss: 5.5320 - 2s/epoch - 157ms/step
Epoch 192/1000
12/12 - 2s - loss: 5.5281 - val_loss: 5.4971 - 2s/epoch - 157ms/step
Epoch 193/1000
12/12 - 2s - loss: 5.7507 - val_loss: 5.4644 - 2s/epoch - 155ms/step
Epoch 194/1000
12/12 - 2s - loss: 5.5429 - val_loss: 5.4348 - 2s/epoch - 156ms/step
Epoch 195/1000
12/12 - 2s - loss: 5.3895 - val_loss: 5.4067 - 2s/epoch - 158ms/step
Epoch 196/1000
12/12 - 2s - loss: 5.3672 - val_loss: 5.3805 - 2s/epoch - 154ms/step
Epoch 197/1000
12/12 - 2s - loss: 5.3995 - val_loss: 5.3562 - 2s/epoch - 155ms/step
Epoch 198/1000
12/12 - 2s - loss: 5.2543 - val_loss: 5.3313 - 2s/epoch - 155ms/step
Epoch 199/1000
12/12 - 2s - loss: 5.3150 - val_loss: 5.3058 - 2s/epoch - 162ms/step
Epoch 200/1000
12/12 - 2s - loss: 5.2522 - val_loss: 5.2814 - 2s/epoch - 155ms/step
Epoch 201/1000
12/12 - 2s - loss: 5.0952 - val_loss: 5.2615 - 2s/epoch - 149ms/step
Epoch 202/1000
12/12 - 2s - loss: 5.0255 - val_loss: 5.2420 - 2s/epoch - 147

Epoch 289/1000
12/12 - 2s - loss: 3.8316 - val_loss: 4.0775 - 2s/epoch - 155ms/step
Epoch 290/1000
12/12 - 2s - loss: 3.6459 - val_loss: 4.0652 - 2s/epoch - 152ms/step
Epoch 291/1000
12/12 - 2s - loss: 3.7176 - val_loss: 4.0523 - 2s/epoch - 163ms/step
Epoch 292/1000
12/12 - 2s - loss: 3.4582 - val_loss: 4.0402 - 2s/epoch - 163ms/step
Epoch 293/1000
12/12 - 2s - loss: 3.6398 - val_loss: 4.0274 - 2s/epoch - 153ms/step
Epoch 294/1000
12/12 - 2s - loss: 3.7120 - val_loss: 4.0153 - 2s/epoch - 146ms/step
Epoch 295/1000
12/12 - 2s - loss: 3.5607 - val_loss: 4.0029 - 2s/epoch - 158ms/step
Epoch 296/1000
12/12 - 2s - loss: 3.5869 - val_loss: 3.9905 - 2s/epoch - 155ms/step
Epoch 297/1000
12/12 - 2s - loss: 3.5811 - val_loss: 3.9784 - 2s/epoch - 154ms/step
Epoch 298/1000
12/12 - 2s - loss: 3.7378 - val_loss: 3.9658 - 2s/epoch - 152ms/step
Epoch 299/1000
12/12 - 2s - loss: 3.6099 - val_loss: 3.9538 - 2s/epoch - 164ms/step
Epoch 300/1000
12/12 - 2s - loss: 3.5513 - val_loss: 3.9410 - 2s/epoch - 165

Epoch 387/1000
12/12 - 2s - loss: 2.5049 - val_loss: 2.9440 - 2s/epoch - 152ms/step
Epoch 388/1000
12/12 - 2s - loss: 2.6266 - val_loss: 2.9344 - 2s/epoch - 155ms/step
Epoch 389/1000
12/12 - 2s - loss: 2.5544 - val_loss: 2.9244 - 2s/epoch - 149ms/step
Epoch 390/1000
12/12 - 2s - loss: 2.6334 - val_loss: 2.9137 - 2s/epoch - 155ms/step
Epoch 391/1000
12/12 - 2s - loss: 2.5173 - val_loss: 2.9040 - 2s/epoch - 152ms/step
Epoch 392/1000
12/12 - 2s - loss: 2.6286 - val_loss: 2.8938 - 2s/epoch - 153ms/step
Epoch 393/1000
12/12 - 2s - loss: 2.6293 - val_loss: 2.8831 - 2s/epoch - 153ms/step
Epoch 394/1000
12/12 - 2s - loss: 2.6786 - val_loss: 2.8730 - 2s/epoch - 173ms/step
Epoch 395/1000
12/12 - 2s - loss: 2.4926 - val_loss: 2.8624 - 2s/epoch - 153ms/step
Epoch 396/1000
12/12 - 2s - loss: 2.4239 - val_loss: 2.8515 - 2s/epoch - 158ms/step
Epoch 397/1000
12/12 - 2s - loss: 2.5641 - val_loss: 2.8421 - 2s/epoch - 155ms/step
Epoch 398/1000
12/12 - 2s - loss: 2.5658 - val_loss: 2.8327 - 2s/epoch - 157

Epoch 485/1000
12/12 - 2s - loss: 1.8146 - val_loss: 2.1187 - 2s/epoch - 153ms/step
Epoch 486/1000
12/12 - 2s - loss: 1.8061 - val_loss: 2.1118 - 2s/epoch - 163ms/step
Epoch 487/1000
12/12 - 2s - loss: 1.9006 - val_loss: 2.1054 - 2s/epoch - 163ms/step
Epoch 488/1000
12/12 - 2s - loss: 1.8064 - val_loss: 2.1017 - 2s/epoch - 157ms/step
Epoch 489/1000
12/12 - 2s - loss: 1.8884 - val_loss: 2.0942 - 2s/epoch - 154ms/step
Epoch 490/1000
12/12 - 2s - loss: 1.8935 - val_loss: 2.0880 - 2s/epoch - 156ms/step
Epoch 491/1000
12/12 - 2s - loss: 1.8173 - val_loss: 2.0806 - 2s/epoch - 153ms/step
Epoch 492/1000
12/12 - 2s - loss: 1.8181 - val_loss: 2.0744 - 2s/epoch - 155ms/step
Epoch 493/1000
12/12 - 2s - loss: 1.8258 - val_loss: 2.0671 - 2s/epoch - 155ms/step
Epoch 494/1000
12/12 - 2s - loss: 1.8484 - val_loss: 2.0609 - 2s/epoch - 155ms/step
Epoch 495/1000
12/12 - 2s - loss: 1.7636 - val_loss: 2.0587 - 2s/epoch - 159ms/step
Epoch 496/1000
12/12 - 2s - loss: 1.8235 - val_loss: 2.0491 - 2s/epoch - 168

Epoch 583/1000
12/12 - 2s - loss: 1.3950 - val_loss: 1.6565 - 2s/epoch - 155ms/step
Epoch 584/1000
12/12 - 2s - loss: 1.4659 - val_loss: 1.6533 - 2s/epoch - 164ms/step
Epoch 585/1000
12/12 - 2s - loss: 1.5038 - val_loss: 1.6496 - 2s/epoch - 161ms/step
Epoch 586/1000
12/12 - 2s - loss: 1.4701 - val_loss: 1.6500 - 2s/epoch - 143ms/step
Epoch 587/1000
12/12 - 2s - loss: 1.4560 - val_loss: 1.6610 - 2s/epoch - 129ms/step
Epoch 588/1000
12/12 - 2s - loss: 1.4680 - val_loss: 1.6429 - 2s/epoch - 171ms/step
Epoch 589/1000
12/12 - 2s - loss: 1.4465 - val_loss: 1.6362 - 2s/epoch - 161ms/step
Epoch 590/1000
12/12 - 2s - loss: 1.4555 - val_loss: 1.6347 - 2s/epoch - 156ms/step
Epoch 591/1000
12/12 - 2s - loss: 1.4728 - val_loss: 1.6298 - 2s/epoch - 155ms/step
Epoch 592/1000
12/12 - 2s - loss: 1.5008 - val_loss: 1.6271 - 2s/epoch - 158ms/step
Epoch 593/1000
12/12 - 1s - loss: 1.3932 - val_loss: 1.6284 - 1s/epoch - 124ms/step
Epoch 594/1000
12/12 - 2s - loss: 1.4522 - val_loss: 1.6215 - 2s/epoch - 156

Epoch 681/1000
12/12 - 2s - loss: 1.3071 - val_loss: 1.4486 - 2s/epoch - 165ms/step
Epoch 682/1000
12/12 - 2s - loss: 1.2893 - val_loss: 1.4530 - 2s/epoch - 127ms/step
Epoch 683/1000
12/12 - 2s - loss: 1.2964 - val_loss: 1.4467 - 2s/epoch - 153ms/step
Epoch 684/1000
12/12 - 2s - loss: 1.2894 - val_loss: 1.4441 - 2s/epoch - 161ms/step
Epoch 685/1000
12/12 - 2s - loss: 1.2218 - val_loss: 1.4486 - 2s/epoch - 128ms/step
Epoch 686/1000
12/12 - 2s - loss: 1.2646 - val_loss: 1.4452 - 2s/epoch - 145ms/step
Epoch 687/1000
12/12 - 2s - loss: 1.2541 - val_loss: 1.4406 - 2s/epoch - 154ms/step
Epoch 688/1000
12/12 - 2s - loss: 1.3145 - val_loss: 1.4410 - 2s/epoch - 131ms/step
Epoch 689/1000
12/12 - 2s - loss: 1.2915 - val_loss: 1.4420 - 2s/epoch - 127ms/step
Epoch 690/1000
12/12 - 2s - loss: 1.2841 - val_loss: 1.4367 - 2s/epoch - 161ms/step
Epoch 691/1000
12/12 - 2s - loss: 1.2930 - val_loss: 1.4356 - 2s/epoch - 156ms/step
Epoch 692/1000
12/12 - 1s - loss: 1.2892 - val_loss: 1.4363 - 1s/epoch - 123

Epoch 779/1000
12/12 - 2s - loss: 1.2811 - val_loss: 1.3784 - 2s/epoch - 153ms/step
Epoch 780/1000
12/12 - 2s - loss: 1.2408 - val_loss: 1.3786 - 2s/epoch - 127ms/step
Epoch 781/1000
12/12 - 2s - loss: 1.3036 - val_loss: 1.3855 - 2s/epoch - 131ms/step
Epoch 782/1000
12/12 - 2s - loss: 1.2502 - val_loss: 1.3778 - 2s/epoch - 161ms/step
Epoch 783/1000
12/12 - 2s - loss: 1.2349 - val_loss: 1.3734 - 2s/epoch - 153ms/step
Epoch 784/1000
12/12 - 2s - loss: 1.2317 - val_loss: 1.3722 - 2s/epoch - 162ms/step
Epoch 785/1000
12/12 - 1s - loss: 1.2689 - val_loss: 1.3731 - 1s/epoch - 123ms/step
Epoch 786/1000
12/12 - 2s - loss: 1.2892 - val_loss: 1.3755 - 2s/epoch - 133ms/step
Epoch 787/1000
12/12 - 1s - loss: 1.2509 - val_loss: 1.3914 - 1s/epoch - 123ms/step
Epoch 788/1000
12/12 - 2s - loss: 1.2931 - val_loss: 1.3771 - 2s/epoch - 129ms/step
Epoch 789/1000
12/12 - 1s - loss: 1.2552 - val_loss: 1.3810 - 1s/epoch - 125ms/step
Epoch 790/1000
12/12 - 2s - loss: 1.2526 - val_loss: 1.3738 - 2s/epoch - 126

Epoch 877/1000
12/12 - 1s - loss: 1.2589 - val_loss: 1.3882 - 1s/epoch - 119ms/step
Epoch 878/1000
12/12 - 2s - loss: 1.2588 - val_loss: 1.3509 - 2s/epoch - 126ms/step
Epoch 879/1000
12/12 - 1s - loss: 1.2139 - val_loss: 1.3666 - 1s/epoch - 122ms/step
Epoch 880/1000
12/12 - 1s - loss: 1.2193 - val_loss: 1.3549 - 1s/epoch - 122ms/step
Epoch 881/1000
12/12 - 1s - loss: 1.2066 - val_loss: 1.3592 - 1s/epoch - 122ms/step
Epoch 882/1000
12/12 - 2s - loss: 1.2456 - val_loss: 1.3641 - 2s/epoch - 131ms/step
Epoch 883/1000
12/12 - 2s - loss: 1.2574 - val_loss: 1.3573 - 2s/epoch - 135ms/step
Epoch 884/1000
12/12 - 1s - loss: 1.2510 - val_loss: 1.3910 - 1s/epoch - 119ms/step
Epoch 885/1000
12/12 - 1s - loss: 1.2377 - val_loss: 1.3744 - 1s/epoch - 122ms/step
Epoch 886/1000
12/12 - 1s - loss: 1.2708 - val_loss: 1.3657 - 1s/epoch - 123ms/step
Epoch 887/1000
12/12 - 1s - loss: 1.2059 - val_loss: 1.3551 - 1s/epoch - 124ms/step
Epoch 888/1000
12/12 - 1s - loss: 1.2515 - val_loss: 1.3511 - 1s/epoch - 116

In [19]:
test_generator = DataGenerator(df_X=X_test, arr_Y=y_test, batch_size=1,mode='test')
STEP_SIZE_VALID = test_generator.n()

predict_result = vit_classifier.predict(test_generator,steps=STEP_SIZE_VALID)
prediction = predict_result

103/103 [==============================] - 6s 36ms/step


In [20]:
results = pd.DataFrame()
results = X_test
results['prediction'] = prediction

In [22]:
results.to_csv('kt_results.csv',index=False)

In [ ]:
vit_classifier.summary()